# Import Libraries and read file

In [1]:
import matplotlib.pyplot as plt
import astropy
import yt
import numpy as np
import os 

# elephant
# hdf5_root = "/srv/data/stratbox_simulations/stratbox_particle_runs/bx5/smd132/sn34/pe300/4pc_resume/4pc"

# s01.leaves
hdf5_root = "/home/joy0921/Desktop/Dataset/hdf5_files"

ds = yt.load(os.path.join(hdf5_root, 'sn34_smd132_bx5_pe300_hdf5_plt_cnt_0206'))

center = [0, 0, 0] * yt.units.pc
arb_center = ds.arr(center, 'code_length')
xlim = 256
ylim = 256
zlim= 256
left_edge = arb_center - ds.quan(xlim // 2, 'pc')
right_edge = arb_center + ds.quan(ylim // 2, 'pc')
obj = ds.arbitrary_grid(left_edge, right_edge, dims=[xlim, ylim, zlim])

/home/joy0921/.local/lib/python3.10/site-packages/yt/sample_data/api.py:144: UserWarning: Storage directory from yt config doesn't exist (currently set to '/does/not/exist'). Current working directory will be used instead.
  warn(


FileNotFoundError: No such file or directory: '/home/joy0921/Desktop/Dataset/hdf5_files/sn34_smd132_bx5_pe300_hdf5_plt_cnt_0206'.